In [1]:
import os
import sys

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
# os.environ["NUMBA_DEBUG_FRONTEND"] = "1"
# os.environ["NUMBA_DEBUG"] = "1"
# os.environ["NUMBA_DEBUGINFO"] = "1"
# os.environ["NUMBA_SHOW_HELP"] = "1"
# os.environ["NUMBA_FULL_TRACEBACKS"] = "1"
# os.environ["NUMBA_DEVELOPER_MODE"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"
# os.environ.get("NUMBA_DEBUG_CACHE")

In [2]:
import numba
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_base import backtest_df_only
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import nb_dos_cart_product
from nb_quantfreedom.strategies.nb_strategy import nb_BacktestInd, nb_Strategy
from numba.experimental import jitclass
from nb_quantfreedom.nb_custom_logger import nb_PrintLogs
from nb_quantfreedom.nb_order_handler.nb_class_helpers import nb_GetMinPrice
from nb_quantfreedom.utils import clear_cache


clear_cache()

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=200,
)

It took 00 mins and 01 seconds to download 200 candles


In [4]:
@jitclass([("svoxel", numba.types.int64[:])])
class FeedBuilder(object):
    def __init__(self, svoxel):
        self.svoxel = svoxel

    def hello(self):
        print("hello world")

    def iterate(self, callback):
        callback(self)


@numba.njit(cache=True)
def callbackFunc(feedBuilder):
    feedBuilder.hello()
    print("callbackFunc")
    # print(feedBuilder.svoxel[1])
    # feedBuilder.svoxel[0] = 900
    # print(feedBuilder.svoxel[0])


@numba.njit(cache=True)
def wrapper():
    fBuilder = FeedBuilder(np.array([1, 10], np.int64))
    fBuilder.iterate(callbackFunc)


wrapper()

[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2617390614.callbackFunc-c6d0d087f8.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2617390614.callbackFunc-c6d0d087f8.py310.1.nbc'
[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2617390614.wrapper-50ed06e03b.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\2617390614.wrapper-50ed06e03b.py310.1.nbc'
hello world
callbackFunc


In [110]:
class nb_GlobalVars:
    def __init__(
        self,
        price_tick_step,
        market_fee_pct,
    ):
        self.price_tick_step = price_tick_step
        self.market_fee_pct = market_fee_pct


global_vars_list = [
    ("price_tick_step", numba.types.int_),
    ("market_fee_pct", numba.types.float_),
]


class TestSLClass:
    def __init__(self):
        pass

    def calc_sl_callback(self, callback):
        pass

    def calc_sl(self, price, lookback, thinger):
        pass


@jitclass(global_vars_list)
class SLCalc(nb_GlobalVars, TestSLClass):
    def calc_sl_callback(self, callback):
        callback(self)

    def calc_sl(self, price, lookback, thinger):
        # print(price)
        # print(lookback)
        return self.price_tick_step * self.market_fee_pct * price - thinger / lookback


@jitclass(global_vars_list)
class newSLCals(nb_GlobalVars, TestSLClass):
    def calc_sl_callback(self, callback):
        callback(self)

    def calc_sl(self, price, lookback, thinger):
        # print("you are dumb")
        # print("hey there")
        return self.price_tick_step * self.market_fee_pct * price / lookback / thinger


@numba.njit(cache=True)
def nb_simulate(sl_calculator: TestSLClass):
    hey = 0.0
    for ind in range(2, 20):
        for dos in range(2, 400):
            for bar in range(2, 1000):
                hey += sl_calculator.calc_sl(price=ind, lookback=dos, thinger=bar)
    print(hey)


@numba.njit(cache=True)
def nb_base():
    sl_calculator = SLCalc(1, 10.5)
    sl_calculator.calc_sl_callback(nb_simulate)
    sl_calculator = newSLCals(10, 10.3)
    sl_calculator.calc_sl_callback(nb_simulate)


nb_base()

[cache] index loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3866667757.nb_base-4967266f39.py310.nbi'
[cache] data loaded from 'C:\\Users\\User\\.ipython\\numba_cache\\3866667757.nb_base-4967266f39.py310.1.nbc'
738194677.8603731
702794.4466513451


In [16]:
@numba.njit(cache=True)
def composition(funcs, x):
    r = x
    for f in funcs[::-1]:
        r = f(r)
    return r


@numba.cfunc("float64(float64)", cache=True)
def a(x):
    return x + 1.0


@numba.njit(cache=True)
def b(x):
    return x * x


composition((a, b), 0.5), 2

[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\397867610.a-3c6352f738.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\397867610.a-3c6352f738.py310.1.nbc'
[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\397867610.composition-8776885b37.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\397867610.composition-8776885b37.py310.1.nbc'
[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\397867610.b-61c07b8194.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\397867610.b-61c07b8194.py310.1.nbc'


(1.25, 2)

In [77]:
import numba
from numba.core import types

@numba.njit()
def f1(x, y):
    return x + y

@numba.njit()
def f2(x, y):
    return x + y

f1(1., 2.)
f2(1., 2.)

f_list = numba.typed.List.empty_list(types.float64(types.float64, types.float64).as_type())
f_list.append(f1)
f_list.append(f2)

@numba.experimental.jitclass([('funcs', types.ListType(types.float64(types.float64, types.float64).as_type()))])
class Handler:

    def __init__(self, funcs):
        self.funcs = funcs

In [78]:
Handler.class_type

jitclass.Handler#1b022991780<funcs:ListType[FunctionType[float64(float64, float64)]]>

In [79]:
import numba
from numba.core import types


class PClass:
    def __init__(self) -> None:
        pass


@numba.experimental.jitclass()
class TestingTEster(PClass):
    def f1(x, y):
        return x + y

    def f2(x, y):
        return x + y


@numba.experimental.jitclass(
    [("funcs", types.ListType(TestingTEster.class_type.instance_type(types.float64(types.float_, types.float_))))]
)
class JITTEST:
    def __init__(self, funcs):
        self.funcs = funcs

In [16]:
# https://numba.discourse.group/t/typed-list-of-jitted-functions-in-jitclass/413/6
import logging
import numba
from numba.core import types


@numba.njit(cache=True)
def log_debug(message):
    print(message+ " this is log debug")
    pass


@numba.njit(cache=True)
def log_info(message):
    print(message + " this is log info")
    pass


@numba.experimental.jitclass(
    [
        ("log_debug", types.void(types.unicode_type).as_type()),
        ("log_info", types.void(types.unicode_type).as_type()),
    ]
)
class PrintTester:
    def __init__(self, log_debug, log_info):
        if log_debug is not None:
            self.log_debug = log_debug
        elif log_debug is None:
            self.log_debug = logging.debug
        self.log_info = log_info


logger = PrintTester(None, log_info)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mUnknown attribute 'debug' of type Module(<module 'logging' from 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python310\\lib\\logging\\__init__.py'>)
[1m
File "C:\Users\User\AppData\Local\Temp\ipykernel_3700\3962179911.py", line 30:[0m
[1m    def __init__(self, log_debug, log_info):
        <source elided>
        elif log_debug is None:
[1m            self.log_debug = logging.debug
[0m            [1m^[0m[0m
[0m
[0m[1mDuring: typing of get attribute at C:\Users\User\AppData\Local\Temp\ipykernel_3700\3962179911.py (30)[0m
[1m
File "C:\Users\User\AppData\Local\Temp\ipykernel_3700\3962179911.py", line 30:[0m
[1m    def __init__(self, log_debug, log_info):
        <source elided>
        elif log_debug is None:
[1m            self.log_debug = logging.debug
[0m            [1m^[0m[0m

[0m[1mDuring: resolving cal

In [22]:
class asdfasdf(object):
    def __init__(self, hey,) -> None:
        self.hey = hey

class Wiwer(asdfasdf):
    def printer(self):
        print("hello")
        

asdfasdf(hey='youou')
Wiwer(hey="yes").printer()

hello
